In [ ]:
#default_exp notebook.show_doc

In [ ]:
# export
from fastai_local.core import *
from fastai_local.imports import *
from fastai_local.data.pipeline import *
from fastai_local.data.external import *
from fastai_local.test import *
from fastai_local.notebook.export import *
import inspect,enum
from IPython.display import Markdown,display

# Show doc
> Functions to show the doc cells in notebooks

In [ ]:
test_cases = [
    listify,    #Basic func
    Pipeline,   #Basic class
    ConfigKey,  #Enum
    compose,    #Func with star args and type annotation
    untar_data, #Func with defaults
    add_docs    #Func with kwargs
]

## Gather the information

In [ ]:
def fn_name(ft):
    "Get the name of `ft`"
    if hasattr(ft, '__name__'):       return ft.__name__
    elif getattr(ft, '_name', False): return ft._name
    else:                             return str(ft).split('.')[-1]

In [ ]:
test_eq([fn_name(o) for o in test_cases], 
        ['listify', 'Pipeline', 'ConfigKey', 'compose', 'untar_data', 'add_docs'])

In [ ]:
def get_anchor(fn):
    "Anchor name associated to `fn`"
    if hasattr(fn,'__qualname__'): return fn.__qualname__
    if inspect.ismethod(fn): return fn_name(fn.__self__) + '.' + fn_name(fn)
    return fn_name(fn)

In [ ]:
test_eq([get_anchor(o) for o in test_cases], 
        ['listify', 'Pipeline', 'ConfigKey', 'compose', 'untar_data', 'add_docs'])
test_eq(get_anchor(Pipeline.composed), 'Pipeline.composed')

The inspect module lets us know quickly if an object is a function or a class but it doesn't distinguish classes and enums.

In [ ]:
def is_enum(cls): 
    "Check if `cls` is an enum or another type of class"
    return cls.__class__ == enum.Enum or cls.__class__ == enum.EnumMeta

In [ ]:
assert is_enum(ConfigKey)
assert not is_enum(Pipeline)

In [ ]:
def get_module_name(ft): 
    "Return the name of the module where `ft` is defined"
    return inspect.getmodule(ft).__name__

In [ ]:
test_eq(_get_mod_name(Pipeline), 'fastai_local.data.pipeline')
test_eq(_get_mod_name(untar_data), 'fastai_local.data.external')
test_eq(_get_mod_name(ConfigKey), 'fastai_local.data.external')

### Links

In [ ]:
def belongs_to_module(t, module_name):
    "Check if `t` belongs to `module_name`."
    if hasattr(t, '__func__'): return belongs_to_module(t.__func__, module_name)
    if not inspect.getmodule(t): return False
    return inspect.getmodule(t).__name__.startswith(module_name)

In [ ]:
assert belongs_to_module(Pipeline, 'fastai_local')
assert belongs_to_module(Tensor, 'torch')
assert belongs_to_module(Path, 'pathlib')
assert belongs_to_module(Pipeline(lambda x:x+2).composed, 'fastai_local')

In [ ]:
PYTORCH_DOCS = 'https://pytorch.org/docs/stable/'

def get_pytorch_link(ft):
    "Returns link to pytorch docs of `ft`."
    name = ft.__name__
    if name == 'device': return f'{PYTORCH_DOCS}tensor_attributes.html#torch-device'
    if name == 'Tensor': return f'{PYTORCH_DOCS}tensors.html#torch-tensor'
    if name.startswith('torchvision'):
        doc_path = get_module_name(ft).replace('.', '/')
        if inspect.ismodule(ft): name = name.replace('.', '-')
        return f'{PYTORCH_DOCS}{doc_path}.html#{name}'
    if name.startswith('torch.nn') and inspect.ismodule(ft): # nn.functional is special case
        nn_link = name.replace('.', '-')
        return f'{PYTORCH_DOCS}nn.html#{nn_link}'
    paths = get_module_name(ft).split('.')
    if len(paths) == 1: return f'{PYTORCH_DOCS}{paths[0]}.html#{paths[0]}.{name}'

    offset = 1 if paths[1] == 'utils' else 0 # utils is a pytorch special case
    doc_path = paths[1+offset]
    if inspect.ismodule(ft): return f'{PYTORCH_DOCS}{doc_path}.html#module-{name}'
    fnlink = '.'.join(paths[:(2+offset)]+[name])
    return f'{PYTORCH_DOCS}{doc_path}.html#{fnlink}'

In [ ]:
#TODO: Fix
#Tensor and device are special cases
test_eq(get_pytorch_link(torch.device), 'https://pytorch.org/docs/stable/tensor_attributes.html#torch-device')
test_eq(get_pytorch_link(Tensor), 'https://pytorch.org/docs/stable/tensors.html#torch-tensor')


In [ ]:
#Might change once the library is renamed fastai.
def _is_fastai_class(ft): return belongs_to_module(ft, 'fastai_source')
def _strip_fastai(s): return re.sub(r'^fastai_local\.', '', s)

In [ ]:
FASTAI_DOCS = '' #TODO: change when live

def get_fn_link(ft):
    "Return function link to notebook documentation of `ft`."
    ft = getattr(ft, '__func__', ft)
    page = '_'.join(source_nb(Pipeline).split('_')[1:]).replace('.ipynb', '.html')
    return f'{FASTAI_DOCS}/{page}#{get_anchor(ft)}'

In [ ]:
test_eq(get_fn_link(Pipeline), '/data_pipeline.html#Pipeline')
test_eq(get_fn_link(Pipeline.composed), '/data_pipeline.html#Pipeline.composed')

In [ ]:
def doc_link(obj, name=None, include_bt:bool=True):
    "Create link to documentation."
    name = name or fn_name(obj)
    if include_bt: name = code_esc(name)
    if belongs_to_module(obj, 'torch') and ('Tensor' not in name): 
        return f'[{name}]({get_pytorch_link(obj)})'
    if _is_fastai_class(obj): return f'[{name}]({get_fn_link(obj)})'
    return name

In [ ]:
doc_link(Pipeline)

'`Pipeline`'

In [ ]:
hasattr(Pipeline(lambda x:x+2).composed, '__func__')

True

In [ ]:
assert is_enum(ConfigKey)
assert not is_enum(Pipeline)

In [ ]:
inspect._VAR_POSITIONAL

<_ParameterKind.VAR_POSITIONAL: 2>

In [ ]:
arg_prefixes = {inspect._VAR_POSITIONAL: '\*', inspect._VAR_KEYWORD:'\*\*'}
def code_esc(s): return f'`{s}`'

In [ ]:
def is_enum(cls): return cls == enum.Enum or cls == enum.EnumMeta

In [ ]:
def link_type(arg_type, arg_name=None, include_bt:bool=True):
    "Create link to documentation."
    arg_name = arg_name or fn_name(arg_type)
    if include_bt: arg_name = code_esc(arg_name)
    return arg_name
    if belongs_to_module(arg_type, 'torch') and ('Tensor' not in arg_name): return f'[{arg_name}]({get_pytorch_link(arg_type)})'
    if is_fastai_class(arg_type): return f'[{arg_name}]({get_fn_link(arg_type)})'
    return arg_name

In [ ]:
def type_repr(t):
    if isinstance(t, partial): return partial_repr(t)
    if hasattr(t, '__forward_arg__'): return link_type(t.__forward_arg__)
    elif getattr(t, '__args__', None):
        args = t.__args__
        if len(args)==2 and args[1] == type(None):
            return f'`Optional`\[{type_repr(args[0])}\]'
        reprs = ', '.join([type_repr(o) for o in args])
        return f'{link_type(t)}\[{reprs}\]'
    else: return link_type(t)

In [ ]:
def format_param(p):
    "Formats function param to `param1:Type=val`. Font weights: param1=bold, val=bold+italic"
    arg_prefix = arg_prefixes.get(p.kind, '') # asterisk prefix for *args and **kwargs
    res = f"**{arg_prefix}{code_esc(p.name)}**"
    if hasattr(p, 'annotation') and p.annotation != p.empty: res += f':{type_repr(p.annotation)}'
    if p.default != p.empty:
        default = getattr(p.default, 'func', p.default)
        default = getattr(default, '__name__', default)
        res += f'=***`{repr(default)}`***'
    return res

In [ ]:
def format_ft_def(func, full_name:str=None)->str:
    "Format and link `func` definition to show in documentation"
    sig = inspect.signature(func)
    name = f'<code>{full_name or func.__name__}</code>'
    fmt_params = [format_param(param) for name,param
                  in sig.parameters.items() if name not in ('self','cls')]
    arg_str = f"({', '.join(fmt_params)})"
    f_name = f"<code>class</code> {name}" if inspect.isclass(func) else name
    return f'{f_name}',f'{name}{arg_str}'

def get_enum_doc(elt, full_name:str)->str:
    "Formatted enum documentation."
    vals = ', '.join(elt.__members__.keys())
    return f'<code>{full_name}</code>',f'<code>Enum</code> = [{vals}]'

def get_cls_doc(elt, full_name:str)->str:
    "Class definition."
    parent_class = inspect.getclasstree([elt])[-1][0][1][0]
    name,args = format_ft_def(elt, full_name)
    if parent_class != object: args += f' :: {link_type(parent_class, include_bt=True)}'
    return name,args

In [ ]:
def show_doc(elt, doc_string=True, full_name=None, title_level=None, alt_doc_string='', markdown=True):
    "Show documentation for element `elt`. Supported types: class, Callable, and enum."
    anchor_id = get_anchor(elt)
    elt = getattr(elt, '__func__', elt)
    full_name = full_name or fn_name(elt)
    if inspect.isclass(elt):
        if is_enum(elt.__class__):   name,args = get_enum_doc(elt, full_name)
        else:                        name,args = get_cls_doc(elt, full_name)
    elif isinstance(elt, Callable):  name,args = format_ft_def(elt, full_name)
    else: raise Exception(f'doc definition not supported for {full_name}')
    #source_link = get_function_source(elt) if is_fastai_class(elt) else ""
    link = 'https://github.com/fastai/fastai/'
    source_link = '<a href="{link}" class="source_link" style="float:right">[source]</a>'
    title_level = ifnone(title_level, 3 if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{anchor_id}" class="doc_header">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n'
    if doc_string and inspect.getdoc(elt):
        doc += inspect.getdoc(elt) + ' '
    if markdown: display(Markdown(doc))
    else: return doc

In [ ]:
show_doc(listify)

<h4 id="listify" class="doc_header"><code>listify</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>listify</code>(**`o`**)

Make `o` a list. 

In [ ]:
show_doc(Pipeline)

<h3 id="Pipeline" class="doc_header"><code>class</code> <code>Pipeline</code><a href="{link}" class="source_link" style="float:right">[source]</a></h3>

> <code>Pipeline</code>(**`tfms`**, **`items`**=***`None`***)

A pipeline of transforms applied to a collection, composed and applied for encode/decode, and setup one at a time 

In [ ]:
show_doc(ConfigKey)

<h3 id="ConfigKey" class="doc_header"><code>ConfigKey</code><a href="{link}" class="source_link" style="float:right">[source]</a></h3>

> <code>Enum</code> = [Data, Archive, Model]

Keys for a path in the fastai config file 

In [ ]:
show_doc(compose)

<h4 id="compose" class="doc_header"><code>compose</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>compose</code>(**\*`funcs`**:`Callable`)

Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to all 

In [ ]:
show_doc(untar_data)

<h4 id="untar_data" class="doc_header"><code>untar_data</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>untar_data</code>(**`url`**, **`fname`**=***`None`***, **`dest`**=***`None`***, **`c_key`**=***`<ConfigKey.Data: 1>`***, **`force_download`**=***`False`***)

Download `url` to `fname` if `dest` doesn't exist, and un-tgz to folder `dest`. 

In [ ]:
show_doc(add_docs)

<h4 id="add_docs" class="doc_header"><code>add_docs</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>add_docs</code>(**\*\*`docs`**)

Copy values from `docs` to `cls` docstrings, and confirm all public methods are documented 

In [ ]:
doc =  f'<h{title_level} id="{anchor_id}" class="doc_header">{name}{source_link}{test_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n'
    doc += f'{test_modal}'